In [1]:
import hashlib
import pandas
import sys
import os
import numpy
from base64 import b64encode
from pprint import pprint

In [2]:
from woldrnaseq import models

In [3]:
HTSW=os.path.expanduser('~diane/proj/htsworkflow')
if HTSW not in sys.path:
    sys.path.append(HTSW)
from htsworkflow.submission import encoded

In [4]:
server = encoded.ENCODED('www.encodeproject.org')
#server = encoded.ENCODED('test.encodedcc.org')
server.load_netrc()
validator = encoded.DCCValidator(server)

In [5]:
uploaded_raw_sheet_filename = 'C1-mouse-forelimb-submission-201907-uploaded-production.xlsx'
uploaded_raw = pandas.ExcelFile(uploaded_raw_sheet_filename)

In [6]:
submitted_experiment = uploaded_raw.parse('Experiment')
submitted_libraries = uploaded_raw.parse('Library')
submitted_replicates = uploaded_raw.parse('Replicate')
submitted_raw_files = uploaded_raw.parse('File')

In [7]:
libraries = models.load_library_tables(['paper_analysis_vdir/library_vdir.tsv'])
experiments = models.load_experiments(['paper_analysis_vdir/experiment_run.tsv'])
libraries.shape

(920, 8)

In [8]:
metadata = submitted_replicates.merge(
    submitted_experiment[['accession', 'aliases:array']].rename({'aliases:array': 'aliases:experiment'}, axis=1),
    left_on='experiment', right_on='aliases:experiment').merge(
        submitted_raw_files.rename({'accession': 'file_accession'}, axis=1),
        left_on='aliases:array', right_on='replicate')
metadata.head()

,uuid,experiment,biological_replicate_number:integer,technical_replicate_number:integer,library,aliases:array,accession,aliases:experiment,file_accession,dataset,...,flowcell_details:json,read_length:integer,file_size:integer,lab,award,file_format,output_type,platform,library_id:skip,replicate
0,bf273ae5-72b5-4237-b03b-a9e68fb56f5c,barbara-wold:c1_e10.5_mouse_limb_run2_17327_A1,1,1,barbara-wold:17327_A1,barbara-wold:17327_A1_b1_t1,ENCSR316KIY,barbara-wold:c1_e10.5_mouse_limb_run2_17327_A1,ENCFF791VYN,barbara-wold:c1_e10.5_mouse_limb_run2_17327_A1,...,"[{""machine"": ""http://jumpgate.caltech.edu/sequ...",50,76952708,barbara-wold,UM1HG009443,fastq,reads,encode:HiSeq2500,17327_A1,barbara-wold:17327_A1_b1_t1
1,bf273ae5-72b5-4237-b03b-a9e68fb56f5c,barbara-wold:c1_e10.5_mouse_limb_run2_17327_A1,1,1,barbara-wold:17327_A1,barbara-wold:17327_A1_b1_t1,ENCSR316KIY,barbara-wold:c1_e10.5_mouse_limb_run2_17327_A1,ENCFF272GSE,barbara-wold:c1_e10.5_mouse_limb_run2_17327_A1,...,"[{""machine"": ""http://jumpgate.caltech.edu/sequ...",50,78307437,barbara-wold,UM1HG009443,fastq,reads,encode:HiSeq2500,17327_A1,barbara-wold:17327_A1_b1_t1
2,af8e4fc1-7a4f-49c3-b5f6-8c4fbcae92de,barbara-wold:c1_e10.5_mouse_limb_run2_17327_A10,1,1,barbara-wold:17327_A10,barbara-wold:17327_A10_b1_t1,ENCSR541RSL,barbara-wold:c1_e10.5_mouse_limb_run2_17327_A10,ENCFF875SEA,barbara-wold:c1_e10.5_mouse_limb_run2_17327_A10,...,"[{""machine"": ""http://jumpgate.caltech.edu/sequ...",50,30471929,barbara-wold,UM1HG009443,fastq,reads,encode:HiSeq2500,17327_A10,barbara-wold:17327_A10_b1_t1
3,af8e4fc1-7a4f-49c3-b5f6-8c4fbcae92de,barbara-wold:c1_e10.5_mouse_limb_run2_17327_A10,1,1,barbara-wold:17327_A10,barbara-wold:17327_A10_b1_t1,ENCSR541RSL,barbara-wold:c1_e10.5_mouse_limb_run2_17327_A10,ENCFF065ITR,barbara-wold:c1_e10.5_mouse_limb_run2_17327_A10,...,"[{""machine"": ""http://jumpgate.caltech.edu/sequ...",50,29889475,barbara-wold,UM1HG009443,fastq,reads,encode:HiSeq2500,17327_A10,barbara-wold:17327_A10_b1_t1
4,2990357c-2cec-4bf7-ac5d-709a14f39ea8,barbara-wold:c1_e10.5_mouse_limb_run2_17327_A11,1,1,barbara-wold:17327_A11,barbara-wold:17327_A11_b1_t1,ENCSR736TCS,barbara-wold:c1_e10.5_mouse_limb_run2_17327_A11,ENCFF753EDK,barbara-wold:c1_e10.5_mouse_limb_run2_17327_A11,...,"[{""machine"": ""http://jumpgate.caltech.edu/sequ...",50,86476491,barbara-wold,UM1HG009443,fastq,reads,encode:HiSeq2500,17327_A11,barbara-wold:17327_A11_b1_t1


In [9]:
metadata.shape

(2074, 23)

In [10]:
metadata_paper = metadata[metadata['library_id:skip'].isin(libraries.index)]
metadata_paper.shape

(1840, 23)

In [12]:
#processed_files = pandas.read_excel('processed_files_test-uploaded2.xlsx', 'File')
processed_files = pandas.read_excel('processed_files_production-resume1.xlsx', 'File')

In [13]:
processed_files.head()

,uuid,accession,aliases:array,assembly,award,dataset,derived_from:array,file_format,file_size:integer,genome_annotation,lab,md5sum,output_type,step_run,submitted_file_name
0,0d4f0bcc-5783-47ff-8726-edd70d25cdbb,ENCFF792RZC,barbara-wold:19906_A5-mm10-M4-male_genome.bam,mm10-minimal,UM1HG009443,/experiments/ENCSR741DDM/,"/file/ENCFF531EHX/,/file/ENCFF506KKD/,/file/EN...",bam,77642556,M4,/labs/barbara-wold/,225757e3e69fa8bf0412cc71c708dd3d,alignments,/analysis-step-runs/ecd38dae-7937-4865-8f82-0a...,/woldlab/castor/home/sau/flowcells/H7CNTBCX2/1...
1,463c07d1-531f-444a-9b9c-6f8526c63b31,ENCFF385LEF,barbara-wold:19906_A5-mm10-M4-male_anno.bam,mm10-minimal,UM1HG009443,/experiments/ENCSR741DDM/,"/file/ENCFF531EHX/,/file/ENCFF506KKD/,/file/EN...",bam,71730686,M4,/labs/barbara-wold/,1da6f195906220e222191910c1b3121f,transcriptome alignments,/analysis-step-runs/ecd38dae-7937-4865-8f82-0a...,/woldlab/castor/home/sau/flowcells/H7CNTBCX2/1...
2,7083665c-c59a-443c-b5e7-511462e792ed,ENCFF603YIU,NaN,mm10-minimal,UM1HG009443,/experiments/ENCSR741DDM/,barbara-wold:19906_A5-mm10-M4-male_genome.bam,bigWig,12492215,M4,/labs/barbara-wold/,f040d64033fea22bc7f27e6abe0fbcf3,signal of all reads,/analysis-step-runs/6e5ac5c3-b9fd-4b10-a13e-47...,/woldlab/castor/home/sau/public_html/H7CNTBCX2...
3,f6985f16-3d28-4d28-ab46-eeb6b349ab3e,ENCFF485MNU,NaN,mm10-minimal,UM1HG009443,/experiments/ENCSR741DDM/,barbara-wold:19906_A5-mm10-M4-male_genome.bam,bigWig,7017411,M4,/labs/barbara-wold/,07cd28255fed50d810465402701034a8,signal of unique reads,/analysis-step-runs/6e5ac5c3-b9fd-4b10-a13e-47...,/woldlab/castor/home/sau/public_html/H7CNTBCX2...
4,4b8da495-f1b7-4b17-96eb-f8ac5f61fa7f,ENCFF810ZVC,NaN,mm10-minimal,UM1HG009443,/experiments/ENCSR741DDM/,"barbara-wold:19906_A5-mm10-M4-male_anno.bam,/f...",tsv,9542941,M4,/labs/barbara-wold/,362e256e66652ebedaabd9c75928f3aa,gene quantifications,/analysis-step-runs/76310d0b-df30-45e6-bd5f-44...,/woldlab/castor/home/sau/flowcells/H7CNTBCX2/1...


In [14]:
processed_files_paper = processed_files[processed_files['dataset'].isin(
    ['/experiments/{}/'.format(x) for x in metadata_paper['accession'].values])]
processed_files_paper.shape

(5520, 15)

In [15]:
publication_id = '/publication-data/ENCSR226XLF/'
publication_set = server.get_json(publication_id)
len(publication_set['related_files'])

2074

In [16]:
publication_set['related_files'][:5]

['/files/ENCFF791VYN/',
 '/files/ENCFF272GSE/',
 '/files/ENCFF875SEA/',
 '/files/ENCFF065ITR/',
 '/files/ENCFF753EDK/']

In [17]:
fastq_ids = ['/files/{}/'.format(x) for x in metadata_paper['file_accession']]
len(fastq_ids)

1840

In [18]:
len(set(publication_set['related_files']).intersection(fastq_ids))

1840

In [19]:
len(set(publication_set['related_files']).difference(fastq_ids))

234

In [20]:
set(publication_set['related_files']).difference(['/files/{}/'.format(x) for x in metadata['file_accession']])

set()

In [21]:
processed_paper_ids = ['/files/{}/'.format(x) for x in processed_files_paper['accession']]

In [22]:
all_file_ids = fastq_ids + processed_paper_ids

In [23]:
len(all_file_ids)

7360

In [24]:
response = None
if set(all_file_ids) != set(publication_set['related_files']):
    print('Need to patch')
    response = server.patch_json(publication_id, {'related_files': all_file_ids})
print(response)

Need to patch
{'status': 'success', '@graph': [{'schema_version': '15', 'references': ['/publications/e0d01543-9965-4edb-933c-778a40575cd9/'], '@id': '/publication-data/ENCSR226XLF/', 'alternate_accessions': [], 'assay_synonyms': ['single cell isolation followed by RNA-seq'], 'assembly': [], 'documents': [], 'internal_tags': [], 'files': ['/files/ENCFF875SEA/', '/files/ENCFF065ITR/', '/files/ENCFF753EDK/', '/files/ENCFF537NIB/', '/files/ENCFF071QVA/', '/files/ENCFF490WYL/', '/files/ENCFF634UGQ/', '/files/ENCFF165IPH/', '/files/ENCFF340AND/', '/files/ENCFF927EVR/', '/files/ENCFF362CCO/', '/files/ENCFF768WOE/', '/files/ENCFF497URQ/', '/files/ENCFF168IIX/', '/files/ENCFF500CNG/', '/files/ENCFF915VIW/', '/files/ENCFF098JRH/', '/files/ENCFF688RZO/', '/files/ENCFF689IXY/', '/files/ENCFF347OAH/', '/files/ENCFF680QNB/', '/files/ENCFF896ZOF/', '/files/ENCFF259FUK/', '/files/ENCFF070BBU/', '/files/ENCFF612KAG/', '/files/ENCFF342TTP/', '/files/ENCFF023NFZ/', '/files/ENCFF767SZM/', '/files/ENCFF57